<a href="https://colab.research.google.com/github/u-masao/vector-search-tutorial-for-japanese/blob/main/Session03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 日本語のためのベクトル検索ハンズオン：Session 3

## 環境構築


### Google Colab 向け Docker インストール

In [ ]:
# Copyright 2024 Drengskapur
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#
# @title {display-mode:"form"}
# @markdown <br/><br/><center><img src="https://cdn.jsdelivr.net/gh/drengskapur/docker-in-colab/assets/docker.svg" height="150"><img src="https://cdn.jsdelivr.net/gh/drengskapur/docker-in-colab/assets/colab.svg" height="150"></center><br/>
# @markdown <center><h1>Docker in Colab</h1></center><center>github.com/drengskapur/docker-in-colab<br/><br/><br/><b>udocker("run hello-world")</b></center><br/>
def udocker_init():
    import os
    if not os.path.exists("/home/user"):
        !pip install udocker > /dev/null
        !udocker --allow-root install > /dev/null
        !useradd -m user > /dev/null
    print(f'Docker-in-Colab 1.1.0\n')
    print(f'Usage:     udocker("--help")')
    print(f'Examples:  https://github.com/indigo-dc/udocker?tab=readme-ov-file#examples')

    def execute(command: str):
        user_prompt = "\033[1;32muser@pc\033[0m"
        print(f"{user_prompt}$ udocker {command}")
        !su - user -c "udocker $command"

    return execute

udocker = udocker_init()

Docker-in-Colab 1.1.0

Usage:     udocker("--help")
Examples:  https://github.com/indigo-dc/udocker?tab=readme-ov-file#examples


In [ ]:
# udocker("pull qdrant/qdrant")

In [ ]:
udocker("run -d -p 127.0.0.1:6333:6333 -p 127.0.0.1:6334:6334 qdrant/qdrant")

user@pc$ udocker run -d -p 127.0.0.1:6333:6333 -p 127.0.0.1:6334:6334 qdrant/qdrant
Error: syntax error at: -d


###  SageMaker 向け docker インストール

(省略)

## パッケージのインストールと読み込み

In [ ]:
!pip install -q qdrant-client japanize_matplotlib sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np

## Qdrant の利用

In [ ]:
from qdrant_client import QdrantClient

qdrant_url = "http://localhost:6333"
qdrant_collection_name = "test_collection"

client = QdrantClient(url=qdrant_url)

In [ ]:
status = client.get_collection(
    collection_name=qdrant_collection_name,
)

print(status)

status = client.delete_collection(
    collection_name=qdrant_collection_name,
)

print(status)

status=<CollectionStatus.GREEN: 'green'> optimizer_status=<OptimizersStatusOneOf.OK: 'ok'> vectors_count=6 indexed_vectors_count=0 points_count=6 segments_count=2 config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=4, distance=<Distance.DOT: 'Dot'>, hnsw_config=None, quantization_config=None, on_disk=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=None), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quantization_config=None) payload_schema={}
True


In [ ]:
from qdrant_client.models import Distance, VectorParams

client.create_collection(
    collection_name=qdrant_collection_name,
    vectors_config=VectorParams(size=4, distance=Distance.DOT),
)

True

In [ ]:
from qdrant_client.models import PointStruct

operation_info = client.upsert(
    collection_name=qdrant_collection_name,
    wait=True,
    points=[
        PointStruct(id=1, vector=[0.05, 0.61, 0.76, 0.74], payload={"city": "Berlin"}),
        PointStruct(id=2, vector=[0.19, 0.81, 0.75, 0.11], payload={"city": "London"}),
        PointStruct(id=3, vector=[0.36, 0.55, 0.47, 0.94], payload={"city": "Moscow"}),
        PointStruct(id=4, vector=[0.18, 0.01, 0.85, 0.80], payload={"city": "New York"}),
        PointStruct(id=5, vector=[0.24, 0.18, 0.22, 0.44], payload={"city": "Beijing"}),
        PointStruct(id=6, vector=[0.35, 0.08, 0.11, 0.44], payload={"city": "Mumbai"}),
    ],
)

print(operation_info)

operation_id=0 status=<UpdateStatus.COMPLETED: 'completed'>


In [ ]:
search_result = client.search(
    collection_name=qdrant_collection_name,
    query_vector=[0.2, 0.1, 0.9, 0.7],
    limit=3
)

_ = [print(x) for x in search_result]

id=4 version=0 score=1.362 payload={'city': 'New York'} vector=None shard_key=None
id=1 version=0 score=1.273 payload={'city': 'Berlin'} vector=None shard_key=None
id=3 version=0 score=1.208 payload={'city': 'Moscow'} vector=None shard_key=None


In [ ]:
from qdrant_client.models import Filter, FieldCondition, MatchValue

search_result = client.search(
    collection_name=qdrant_collection_name,
    query_vector=[0.2, 0.1, 0.9, 0.7],
    query_filter=Filter(
        must=[FieldCondition(key="city", match=MatchValue(value="London"))]
    ),
    with_payload=True,
    limit=3,
)

_ = [print(x) for x in search_result]

id=2 version=0 score=0.871 payload={'city': 'London'} vector=None shard_key=None
